In [1]:
"""Tests for the module shapley.py for the test case in section 3.2 in Iooss and 
Prieur (2019): Linear model with two Gaussian inputs.

"""
import chaospy as cp
import numpy as np
import pandas as pd
from numpy.testing import assert_array_almost_equal as aaae

from econsa_shapley import _r_condmvn
from econsa_shapley import get_shapley
from simulation_of_variance import simulate_variance

In [50]:
def test_get_shapley_linear_two_inputs():
    def linear_model(x):
        beta = np.array([[beta_1], [beta_2]])
        return x.dot(beta)

    def x_all(n):
        return cp.MvNormal(mean, cov).sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        if subsetj_conditional is None:
            cov_int = np.array(cov).take(subset_j, axis=1)[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            return distribution.sample(n)
        else:
            return _r_condmvn(
                n,
                mean=mean,
                cov=cov,
                dependent_ind=subset_j,
                given_ind=subsetj_conditional,
                x_given=xjc,
            )

    np.random.seed(1234)
    beta_1 = 1.3
    beta_2 = 1.5
    var_1 = 16
    var_2 = 4
    rho = 0.3

    # Calculate analytical Shapley effects.
    component_1 = beta_1 ** 2 * var_1
    component_2 = beta_2 ** 2 * var_2
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_2)
    var_y = component_1 + 2 * covariance * beta_1 * beta_2 + component_2
    share = 0.5 * (rho**2)
    true_shapley_1 = (component_1 * (1 - share) + covariance * beta_1 * beta_2 + component_2 * share)/var_y
    true_shapley_2 = (component_2 * (1 - share) + covariance * beta_1 * beta_2 + component_1 * share)/var_y

    n_inputs = 2
    mean = np.zeros(n_inputs)
    cov = np.array(
        [[var_1, covariance],
        [covariance, var_2]]
        )
    method = "exact"
    n_perms = None
    n_output = 10 ** 7
    n_outer = 10 ** 5
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [true_shapley_1, true_shapley_2],
            [0, 0],
            [true_shapley_1, true_shapley_2],
            [true_shapley_1, true_shapley_2],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        linear_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated["Shapley effects"], expected["Shapley effects"], 6)


test_get_shapley_linear_two_inputs()

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 2 / 2 (100%)
Max absolute difference: 5.83247836e-05
Max relative difference: 0.00018272
 x: array([0.680856, 0.319144])
 y: array([0.680797, 0.319203])

In [49]:
# Check whether model variance is precisely estimated. Use simulate_variance.
# Define model.
def linear_model(x):
    beta = np.array([[beta_1], [beta_2]])
    return x.dot(beta)

def x_all(n):
    return cp.MvNormal(mean, cov).sample(n)
# Setup model inputs
np.random.seed(123)
beta_1 = 1.3
beta_2 = 1.5
var_1 = 16
var_2 = 4
rho = 0.3

# Calculate analytical Shapley effects.
component_1 = beta_1 ** 2 * var_1
component_2 = beta_2 ** 2 * var_2
covariance = rho * np.sqrt(var_1) * np.sqrt(var_2)
var_y = component_1 + 2 * covariance * beta_1 * beta_2 + component_2

n_inputs = 2
n_output = 10 ** 7
mean = np.zeros(n_inputs)
cov = np.array(
    [[var_1, covariance],
    [covariance, var_2]]
    )

# Simulate variance and compare to analytical model variance.
estimated_variance = simulate_variance(model=linear_model,
                                      cov=cov,
                                      mean=mean,
                                      n_sim=n_output)

print(estimated_variance, var_y)

45.39942467549999 45.400000000000006


In [36]:
# Further function inputs.
def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov).take(subset_j, axis=1)[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
        )
method='exact'
n_perms=None
n_inner = 10 ** 2
n_outer_range = np.array([10 ** 3, 10 ** 4, 10 ** 6, 2 * 10 ** 6, 3 * 10 ** 6, 4 * 10 ** 6])
n_outer_range = np.array([10 ** 3, 10 ** 4, 10 ** 5])

In [37]:
%%time
for i in n_outer_range:
    n_outer = i
    out = get_shapley(
        method,
        linear_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )
    print(out)

    Shapley effects  std. errors    CI_min    CI_max
X1         0.684211     0.094051  0.499871  0.868551
X2         0.315789     0.094051  0.131449  0.500129
    Shapley effects  std. errors    CI_min    CI_max
X1         0.684734     0.096309  0.495968  0.873501
X2         0.315266     0.096309  0.126499  0.504032
    Shapley effects  std. errors    CI_min    CI_max
X1         0.679913     0.099102  0.485674  0.874153
X2         0.320087     0.099102  0.125847  0.514326
Wall time: 3min 30s


In [38]:
%%time
n_outer_range = np.array([10 ** 6])
for i in n_outer_range:
    n_outer = i
    out = get_shapley(
        method,
        linear_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )
    print(out)

    Shapley effects  std. errors    CI_min    CI_max
X1         0.680707     0.098198  0.488239  0.873176
X2         0.319293     0.098198  0.126824  0.511761
Wall time: 33min 3s


In [51]:
%%time
n_outer = 10 ** 5
n_inner = 10 ** 2
n_ouput = 10 ** 7
get_shapley(
    method,
    linear_model,
    x_all,
    x_cond,
    n_perms,
    n_inputs,
    n_output,
    n_outer,
    n_inner,
    )

Wall time: 3min 8s


,Shapley effects,std. errors,CI_min,CI_max
X1,0.680564,0.098387,0.487725,0.873404
X2,0.319436,0.098387,0.126596,0.512275
